In [ ]:
pip install transformers

In [ ]:
from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))

[{'label': 'POSITIVE', 'score': 0.9998704791069031}]


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
file = pd.read_csv("/content/drive/My Drive/CMPE_256/tweets_10-24-2020.csv")
file = file.drop(["isRetweet","isDeleted","device","favorites","retweets"], axis=1)

In [ ]:
file.head()

,id,text,date
0,98454970654916608,Republicans and Democrats have both created ou...,2011-08-02 18:07:48
1,1234653427789070336,I was thrilled to be back in the Great city of...,2020-03-03 01:34:50
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,2020-01-17 03:22:47
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,2020-09-12 20:10:58
4,1218159531554897920,RT @MZHemingway: Very friendly telling of even...,2020-01-17 13:13:59


In [ ]:
def removing_stopwords(text):
    stop_words = set(stopwords.words('english'))
    stop = [x.lower() for x in stop_words]
    word_token = []
    for word in text.split(" "):
      if not word.lower() in stopwords.words():
        if word.isupper() and len(word) > 3:
          word_token.insert(0,word.capitalize())
        else:
          word_token.insert(0,word)
    return [word for word in text.split(" ") if not word.lower() in stopwords.words()]

In [ ]:
i = 0
for label, row in file.iterrows():
  no_dot_tweet = re.sub(r'\.', '', row["text"])
  tweet_after_words_list = removing_stopwords(no_dot_tweet)
  final_tweet = re.sub(r'http\S+|www.\S+|#\S+|@\S+', '', ' '.join(tweet_after_words_list))
  
  ner_results = nlp(final_tweet)
  stack = []
  for x in ner_results:
    if x['score'] > 0.98 and (x['entity'] == "B-LOC" or x['entity'] == "I-LOC"):
      if (x['entity'] == "B-LOC" and len(stack) == 0) or x['entity'] == "I-LOC":
        stack.insert(0,x['word'])
      elif x['entity'] == "B-LOC" and len(stack) > 0:
        y = ""
        while len(stack) > 0:
          z = stack.pop()
          y = y + z + " "
        stack.insert(0,x['word'])
        if y not in file.columns:
          file[y] = pd.Series(0, index=file.index)
        file.loc[file.id == row['id'], y] = 1
  y = ""
  while len(stack) > 0:
    z = stack.pop()
    y = y+z+" "     
  if y not in file.columns:
    file[y] = pd.Series(0, index=file.index)
  file.loc[file.id == row['id'], y] = 1

In [ ]:
file.drop(columns=[''])

,id,text,date,Charlotte,North Carolina,Florida,China,United States,Sudan,Israel,United Arab Emirates Bahrain,Ohio,New Orleans,Louisiana,America,Gujarat,Arkansas,Seattle,New York,Arizona,Center,Carolina,USA,NYC,Pennsylvania,Indonesia,West Virginia,Texas,Colorado,US,Virginia,Georgia,New York City,Portland,Tel Aviv Jerusalem,Nevada,Oklahoma,Lebanon,Mexico,Fort My Arlington,...,Chi ##ba,Mont ##ville,Mont ##ours ##ville,Great Lakes,Mars,United States China,Panama City Beach,Rush Dakota,Pleasant Baptist,St Mary Baptist Church Union,Potomac,Po ##way,Sri Lanka,Côte d ' I ##ire,Bethlehem,Arab Republic,##xi ##co,Cale ##xi ##co,Mexico Border,North &,Go Heights,Capitol Hill,Dominican Republic,Jamaica,Saint Lucia,Israel Go ##lan Heights,Northwest,Suffolk,China Mexico,USA ##town,San Diego Paso,Wall Country,Alaska ##end ##orf,##oi,##iba ##chi,Park ##land,Southern Border Honduras,Chattanooga Tennessee,White House Garden,Egypt East
0,98454970654916608,Republicans and Democrats have both created ou...,2011-08-02 18:07:48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1234653427789070336,I was thrilled to be back in the Great city of...,2020-03-03 01:34:50,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,2020-01-17 03:22:47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,2020-09-12 20:10:58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1218159531554897920,RT @MZHemingway: Very friendly telling of even...,2020-01-17 13:13:59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54437,1319484210101379072,RT @EliseStefanik: President @realDonaldTrump ...,2020-10-23 03:42:05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54438,1319444420861829121,RT @TeamTrump: LIVE: Presidential Debate #Deba...,2020-10-23 01:03:58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54439,1319384118849949702,Just signed an order to support the workers of...,2020-10-22 21:04:21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54440,1319345719829008387,Suburban women want Safety &amp; Security. Joe...,2020-10-22 18:31:46,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
file.columns

Index(['id', 'text', 'date', '', 'Charlotte ', 'North Carolina ', 'Florida ',
       'China ', 'United States ', 'Sudan ',
       ...
       'San Diego Paso ', 'Wall Country ', 'Alaska ##end ##orf ', '##oi ',
       '##iba ##chi ', 'Park ##land ', 'Southern Border Honduras ',
       'Chattanooga Tennessee ', 'White House Garden ', 'Egypt East '],
      dtype='object', length=1228)

In [ ]:
df = file.copy()

In [ ]:
df.head()

,id,text,date,,Charlotte,North Carolina,Florida,China,United States,Sudan,Israel,United Arab Emirates Bahrain,Ohio,New Orleans,Louisiana,America,Gujarat,Arkansas,Seattle,New York,Arizona,Center,Carolina,USA,NYC,Pennsylvania,Indonesia,West Virginia,Texas,Colorado,US,Virginia,Georgia,New York City,Portland,Tel Aviv Jerusalem,Nevada,Oklahoma,Lebanon,Mexico,...,Chi ##ba,Mont ##ville,Mont ##ours ##ville,Great Lakes,Mars,United States China,Panama City Beach,Rush Dakota,Pleasant Baptist,St Mary Baptist Church Union,Potomac,Po ##way,Sri Lanka,Côte d ' I ##ire,Bethlehem,Arab Republic,##xi ##co,Cale ##xi ##co,Mexico Border,North &,Go Heights,Capitol Hill,Dominican Republic,Jamaica,Saint Lucia,Israel Go ##lan Heights,Northwest,Suffolk,China Mexico,USA ##town,San Diego Paso,Wall Country,Alaska ##end ##orf,##oi,##iba ##chi,Park ##land,Southern Border Honduras,Chattanooga Tennessee,White House Garden,Egypt East
0,98454970654916608,Republicans and Democrats have both created ou...,2011-08-02 18:07:48,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1234653427789070336,I was thrilled to be back in the Great city of...,2020-03-03 01:34:50,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,2020-01-17 03:22:47,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,2020-09-12 20:10:58,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1218159531554897920,RT @MZHemingway: Very friendly telling of even...,2020-01-17 13:13:59,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
for x in file.columns:
  text = re.findall(r"[^a-zA-Z\s]+", x)
  if text != []:
    df.drop(columns=[x],inplace =True)

In [ ]:
df.head()

,id,text,date,,Charlotte,North Carolina,Florida,China,United States,Sudan,Israel,United Arab Emirates Bahrain,Ohio,New Orleans,Louisiana,America,Gujarat,Arkansas,Seattle,New York,Arizona,Center,Carolina,USA,NYC,Pennsylvania,Indonesia,West Virginia,Texas,Colorado,US,Virginia,Georgia,New York City,Portland,Tel Aviv Jerusalem,Nevada,Oklahoma,Lebanon,Mexico,...,Gulf Coast,St Louis Park,North Carolina States,Washington Monument,AM,New York State York,New Jersey York,Florida Office,Heights,United States Poland,Gulf Oman,Mexico USA,NE,United Kingdom States,Great Lakes,Mars,United States China,Panama City Beach,Rush Dakota,Pleasant Baptist,St Mary Baptist Church Union,Potomac,Sri Lanka,Bethlehem,Arab Republic,Mexico Border,Go Heights,Capitol Hill,Dominican Republic,Jamaica,Saint Lucia,Northwest,Suffolk,China Mexico,San Diego Paso,Wall Country,Southern Border Honduras,Chattanooga Tennessee,White House Garden,Egypt East
0,98454970654916608,Republicans and Democrats have both created ou...,2011-08-02 18:07:48,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1234653427789070336,I was thrilled to be back in the Great city of...,2020-03-03 01:34:50,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,2020-01-17 03:22:47,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,2020-09-12 20:10:58,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1218159531554897920,RT @MZHemingway: Very friendly telling of even...,2020-01-17 13:13:59,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.to_csv("/content/drive/My Drive/CMPE_256/NER_location.csv",index=False)

In [ ]:
xml = pd.read_csv("/content/drive/My Drive/CMPE_256/NER_location.csv")

In [ ]:
xml.columns

Index(['id', 'text', 'date', 'Unnamed: 3', 'Charlotte ', 'North Carolina ',
       'Florida ', 'China ', 'United States ', 'Sudan ',
       ...
       'Saint Lucia ', 'Northwest ', 'Suffolk ', 'China Mexico ',
       'San Diego Paso ', 'Wall Country ', 'Southern Border Honduras ',
       'Chattanooga Tennessee ', 'White House Garden ', 'Egypt East '],
      dtype='object', length=914)